In [82]:
import pandas as pd
import numpy as np
import gensim
import spacy
import string
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
import logging
import re
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
#importing data
cases = pd.read_csv('/Users/samk/Box/Riverstone 2 (Phoebe, Nick, Jared, John)/finalcase.csv')

In [6]:
cases.head()

,Unnamed: 0,citation_num,citation_type,court_id,court_name,year,month,day,defendant,defendants_attorneys,...,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter
0,1,144 N.H. 131,official,8797,New Hampshire Supreme Court,1999,8.0,30.0,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",...,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports
1,2,144 N.H. 13,official,8797,New Hampshire Supreme Court,1999,7.0,13.0,Bennett,"Janice S. Peterson, public defender, of Keene,...",...,New Hampshire,The State of New Hampshire v. Eric Bennett,State v. Bennett,"['BRODERICK, j.', 'THAYER, J.,']","['BRODERICK, j.\nAfter a jury trial in Superio...","['majority', 'concurrence']",['The State of New Hampshire v. Eric Bennett'],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports
2,3,144 N.H. 138,official,8797,New Hampshire Supreme Court,1999,8.0,30.0,NaN,"Shaines & McEachern, P.A., of Portsmouth (Paul...",...,New Hampshire,Roberge’s Case,Roberge’s Case,['PER CURIAM.'],['MEMORANDUM OPINION\nPER CURIAM.\nThe Supreme...,['majority'],['Roberge’s Case'],Roberge’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports
3,4,144 N.H. 107,official,8797,New Hampshire Supreme Court,1999,8.0,3.0,NaN,"Wiggin & Nourie, P.A., of Manchester (Scott A....",...,New Hampshire,"Appeal of Donald E. Savage, Jr. (New Hampshire...",Appeal of Savage,"['BRODERICK, J.']","['BRODERICK, J.\nThe petitioner, Donald E. Sav...",['majority'],"['Appeal of Donald E. Savage, Jr. (New Hampshi...",Appeal of Savage,"Fitzgerald, & Sessler, P.A., of Laconia (Shawn...",New Hampshire Reports
4,5,144 N.H. 44,official,8797,New Hampshire Supreme Court,1999,7.0,21.0,NaN,"Philip T. McLaughlin, attorney general (Dougla...",...,New Hampshire,"Appeal of William H. Morgan, R.PH. (New Hampsh...",Appeal of Morgan,"['HORTON, J.']","[""HORTON, J.\nThe petitioner, William H. Morga...",['majority'],"['Appeal of William H. Morgan, R.PH. (New Hamp...",Appeal of Morgan,"William, II. Loftus, P.C., of Lebanon (William...",New Hampshire Reports


In [95]:
#Lowercasing the text for word search consistency
cases['opinion_text_clean'] = cases['opinion_text'].apply(lambda x : x.lower())

In [96]:
cases.head()

,Unnamed: 0,citation_num,citation_type,court_id,court_name,year,month,day,defendant,defendants_attorneys,...,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter,opinion_text_clean
0,1,144 N.H. 131,official,8797,New Hampshire Supreme Court,1999,8.0,30.0,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",...,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,"[""johnson, j.\nthe supreme court committee on ..."
1,2,144 N.H. 13,official,8797,New Hampshire Supreme Court,1999,7.0,13.0,Bennett,"Janice S. Peterson, public defender, of Keene,...",...,The State of New Hampshire v. Eric Bennett,State v. Bennett,"['BRODERICK, j.', 'THAYER, J.,']","['BRODERICK, j.\nAfter a jury trial in Superio...","['majority', 'concurrence']",['The State of New Hampshire v. Eric Bennett'],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports,"['broderick, j.\nafter a jury trial in superio..."
2,3,144 N.H. 138,official,8797,New Hampshire Supreme Court,1999,8.0,30.0,NaN,"Shaines & McEachern, P.A., of Portsmouth (Paul...",...,Roberge’s Case,Roberge’s Case,['PER CURIAM.'],['MEMORANDUM OPINION\nPER CURIAM.\nThe Supreme...,['majority'],['Roberge’s Case'],Roberge’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['memorandum opinion\nper curiam.\nthe supreme...
3,4,144 N.H. 107,official,8797,New Hampshire Supreme Court,1999,8.0,3.0,NaN,"Wiggin & Nourie, P.A., of Manchester (Scott A....",...,"Appeal of Donald E. Savage, Jr. (New Hampshire...",Appeal of Savage,"['BRODERICK, J.']","['BRODERICK, J.\nThe petitioner, Donald E. Sav...",['majority'],"['Appeal of Donald E. Savage, Jr. (New Hampshi...",Appeal of Savage,"Fitzgerald, & Sessler, P.A., of Laconia (Shawn...",New Hampshire Reports,"['broderick, j.\nthe petitioner, donald e. sav..."
4,5,144 N.H. 44,official,8797,New Hampshire Supreme Court,1999,7.0,21.0,NaN,"Philip T. McLaughlin, attorney general (Dougla...",...,"Appeal of William H. Morgan, R.PH. (New Hampsh...",Appeal of Morgan,"['HORTON, J.']","[""HORTON, J.\nThe petitioner, William H. Morga...",['majority'],"['Appeal of William H. Morgan, R.PH. (New Hamp...",Appeal of Morgan,"William, II. Loftus, P.C., of Lebanon (William...",New Hampshire Reports,"[""horton, j.\nthe petitioner, william h. morga..."


In [26]:
#concatenating opinion text
raw_text= []
for i in range(0,len(cases)):
    text = ''.join(cases['opinion_text'][i])
    raw_text.append(text)

In [48]:
#Minimal preprocessing of the text
raw_text_prep = []
for i in range(0, len(raw_text)):
    clean = gensim.utils.simple_preprocess(raw_text[i])
    raw_text_prep.append(clean)
    

In [62]:
#Initializing word2vec model
path = get_tmpfile("word2vec.model")
model = Word2Vec(raw_text_prep, size= 100, window=5, min_count=1, workers=6)
model.save("word2vec.model")

2019-03-05 20:19:29,320 : INFO : collecting all words and their counts
2019-03-05 20:19:29,321 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-05 20:19:31,071 : INFO : PROGRESS: at sentence #10000, processed 16436411 words, keeping 73046 word types
2019-03-05 20:19:32,945 : INFO : collected 106125 word types from a corpus of 34279962 raw words and 19726 sentences
2019-03-05 20:19:32,946 : INFO : Loading a fresh vocabulary
2019-03-05 20:19:33,495 : INFO : effective_min_count=1 retains 106125 unique words (100% of original 106125, drops 0)
2019-03-05 20:19:33,496 : INFO : effective_min_count=1 leaves 34279962 word corpus (100% of original 34279962, drops 0)
2019-03-05 20:19:33,722 : INFO : deleting the raw counts dictionary of 106125 items
2019-03-05 20:19:33,724 : INFO : sample=0.001 downsamples 38 most-common words
2019-03-05 20:19:33,725 : INFO : downsampling leaves estimated 25492617 word corpus (74.4% of prior 34279962)
2019-03-05 20:19:33,983 : IN

2019-03-05 20:20:19,740 : INFO : EPOCH 4 - PROGRESS: at 95.98% examples, 2238984 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:20:20,051 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-03-05 20:20:20,053 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-03-05 20:20:20,055 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-05 20:20:20,056 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-05 20:20:20,059 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-05 20:20:20,063 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-05 20:20:20,065 : INFO : EPOCH - 4 : training on 34279962 raw words (25330376 effective words) took 11.3s, 2232449 effective words/s
2019-03-05 20:20:21,068 : INFO : EPOCH 5 - PROGRESS: at 7.72% examples, 2166631 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:20:22,072 : INFO : EPOCH 5 - PROGRESS: at 17.69% examples, 2209829 wor

In [63]:
#Training the word2vec model
model.train(raw_text_prep, total_examples=len(raw_text_prep), epochs=10)

2019-03-05 20:21:23,274 : WARNING : Effective 'alpha' higher than previous training cycles
2019-03-05 20:21:23,275 : INFO : training model with 6 workers on 106125 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-03-05 20:21:24,279 : INFO : EPOCH 1 - PROGRESS: at 7.91% examples, 2221091 words/s, in_qsize 10, out_qsize 1
2019-03-05 20:21:25,281 : INFO : EPOCH 1 - PROGRESS: at 17.49% examples, 2195138 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:21:26,285 : INFO : EPOCH 1 - PROGRESS: at 23.03% examples, 2247488 words/s, in_qsize 10, out_qsize 1
2019-03-05 20:21:27,289 : INFO : EPOCH 1 - PROGRESS: at 34.37% examples, 2253627 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:21:28,294 : INFO : EPOCH 1 - PROGRESS: at 48.58% examples, 2250648 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:21:29,298 : INFO : EPOCH 1 - PROGRESS: at 54.92% examples, 2254088 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:21:30,298 : INFO : EPOCH 1 - PROGRESS: at 61.63% exampl

2019-03-05 20:22:11,122 : INFO : EPOCH 5 - PROGRESS: at 23.32% examples, 2275883 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:22:12,125 : INFO : EPOCH 5 - PROGRESS: at 34.86% examples, 2271765 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:22:13,127 : INFO : EPOCH 5 - PROGRESS: at 48.95% examples, 2264760 words/s, in_qsize 10, out_qsize 1
2019-03-05 20:22:14,130 : INFO : EPOCH 5 - PROGRESS: at 55.15% examples, 2267775 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:22:15,132 : INFO : EPOCH 5 - PROGRESS: at 61.64% examples, 2260660 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:22:16,134 : INFO : EPOCH 5 - PROGRESS: at 66.13% examples, 2262348 words/s, in_qsize 10, out_qsize 1
2019-03-05 20:22:17,138 : INFO : EPOCH 5 - PROGRESS: at 72.09% examples, 2259200 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:22:18,139 : INFO : EPOCH 5 - PROGRESS: at 88.13% examples, 2254440 words/s, in_qsize 10, out_qsize 1
2019-03-05 20:22:19,139 : INFO : EPOCH 5 - PROGRESS: at 96.63% examples, 2254000

2019-03-05 20:23:00,007 : INFO : EPOCH 9 - PROGRESS: at 61.68% examples, 2263954 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:23:01,010 : INFO : EPOCH 9 - PROGRESS: at 65.74% examples, 2241249 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:23:02,010 : INFO : EPOCH 9 - PROGRESS: at 71.43% examples, 2242919 words/s, in_qsize 10, out_qsize 0
2019-03-05 20:23:03,014 : INFO : EPOCH 9 - PROGRESS: at 87.44% examples, 2241714 words/s, in_qsize 11, out_qsize 0
2019-03-05 20:23:04,018 : INFO : EPOCH 9 - PROGRESS: at 96.11% examples, 2242792 words/s, in_qsize 12, out_qsize 0
2019-03-05 20:23:04,274 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-03-05 20:23:04,275 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-03-05 20:23:04,277 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-05 20:23:04,278 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-05 20:23:04,281 : INFO : worker thread finished; awai

(253309938, 342799620)

In [323]:
# function returning 3 words based on their similarity with the word provided
def keywords(word):
    keyword = model.wv.most_similar(positive = word)
    keyword_list = [x[0] for x in keyword[:3]]
    
    return keyword_list
    

In [363]:
# keywords in sentences to be extracted along with their word2vec similarities
list_keywords = ['err','conclude','affirm','dismiss','rule','appeal',
                 'charge','convict', 'hold', 'reverse', 'dissent', 'grant','remand', 'concur',
                ]


In [370]:
# function extracting sentences containing keywords in the list_keywords and their 3 most similar neighbors in the word2vec model
def keyword_extract(opinion_text, list_keywords):
    patterns = []
    for i in range(len(list_keywords)):
        keyword = keywords(list_keywords[i])
        patterns.append(keyword)
    keys = []
    for list in patterns:
        for i in range(len(list)):
            words = re.findall(r'([^.]*'+ re.escape(list[i])+'*[^.]*)', opinion_text)#, re.MULTILINE)
            keys.append(words)

    keys = [x for x in keys if x != []] 
    joined = [','.join(elem) for elem in keys]
    joined_text = [elem.replace('\\n','') for elem in joined]
    return joined_text
    

In [371]:
keyword_extract(cases['opinion_text'][2], list_keywords)

[' This court referred the matter to a Judicial Referee (Temple, J',
 ' We uphold the referee’s determination that no violation of New Hampshire Rule of Professional Conduct (Rule) 3',
 'In professional conduct matters, although we defer to the referee’s factual findings if supported by the record, see Wehringer’s Case, 130 N,The record in this case supports the findings of the referee that the respondent was acting in an individual capacity rather than in a lawyer-client relationship and that she did not engage in any intentional misrepresentation which would subject an attorney to discipline when acting in a personal capacity,The referee’s finding, by clear and convincing evidence, that the respondent, under the unique circumstances of this case, did not violate Rule 8, Therefore, we adopt the referee’s findings and dismiss the petition',
 ' We uphold the referee’s determination that no violation of New Hampshire Rule of Professional Conduct (Rule) 3',
 '2d 252, 257 (1988), we retain

In [367]:
case2

[') erred by refusing to instruct the jury on the defense of accident',
 ' after the defendant entered his car, officer elwell reached through the open driver-side window with the hand holding his pistol, he could not do so because the defendant was holding his arm',
 ' you must find the defendant not guilty if you find that the defendant’s acts were accidental, the state has the burden of proving beyond a reasonable doubt that the defendant did not act accidentally; that is, the state must prove that the defendant acted_(purposely, knowingly, recklessly, or negligently), which i have already defined for you, we have held that a defendant’s requested jury instruction on an accident defense must be granted, however, if there is some evidence to support a rational finding in favor of that defense,the defendant is entitled to a jury instruction on the defense of accident only if there is some evidence to support a rational finding in favor of that defense, based on this testimony, the tri